In [3]:
# use env house-prices-advanced-regression-technique

In [2]:
import pandas as pd
from sklearn import linear_model

In [73]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

C:\Users\Joslyn\AppData\Local\Temp\ipykernel_21532\3180332355.py:1: DtypeWarning: Columns (435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506) have mixed types. Specify dtype option on import or set low_memory=False.
  train = pd.read_csv('train.csv')


### EDA

In [85]:
train.isna().sum()


id                    0
p_num                 0
time                  0
bg-5:55           27254
bg-5:50           18491
                  ...  
activity-0:15    174293
activity-0:10    174287
activity-0:05    174271
activity-0:00    174287
bg+1:00               0
Length: 508, dtype: int64

In [82]:
hr_cols = train.filter(regex='hr').columns
hr_vals = train.loc[0, hr_cols]
with open('hr.txt', 'w') as f:
    f.write(hr_vals.to_string())

In [81]:
activity_cols = train.filter(regex='activity').columns
activity_vals = train.loc[0, activity_cols]
with open('activity.txt', 'w') as f:
    f.write(activity_vals.to_string())

In [79]:
cals_cols = train.filter(regex='cals').columns
cals_vals = train.loc[0, cals_cols]
with open('cals.txt', 'w') as f:
    f.write(cals_vals.to_string())

fill cals with 0

In [ ]:
# fill Nan with 0, or consider mean
carb_cols = train.filter(regex='carbs').columns
carb_vals = train.loc[0, carb_cols]
with open('carbs.txt', 'w') as f:
    f.write(carb_vals.to_string())

fill carbs with 0

In [ ]:
insulin_cols = train.filter(regex='insulin').columns
train_partial_fill = train[insulin_cols].ffill(axis=1).bfill(axis=1)
insulin_vals = train.loc[0, insulin_cols]
with open('insulin.txt', 'w') as f:
    f.write(insulin_vals.to_string())

fill NaN insulin cols with 0 first, then try mean

In [87]:
bg_cols = train.filter(regex='bg').columns
train_partial_fill = train[bg_cols].ffill(axis=1).bfill(axis=1)
bg_vals = train_partial_fill.loc[0, bg_cols]
with open('row_0_ffill.txt', 'w') as f:
    f.write(bg_vals.to_string())

take bg columns, average them over hours, yield 6 features

In [24]:
train.shape

(177024, 508)

In [22]:
numerical_features = train.select_dtypes(include=[int, float])
categorical_features = train.select_dtypes(include=[object])

### Feature Creation

In [101]:
target = ['bg+1:00']
y_train = train[target]

In [ ]:
train_features = train.drop(target, axis=1)

In [ ]:
bg_cols = train_features.filter(regex='bg').columns
train_features[bg_cols] = train_features[bg_cols].ffill(axis=1).bfill(axis=1)

In [ ]:
bg_0 = train_features.filter(regex='bg-0')
test['bg_zero'] = train_features[bg_0.columns].mean(axis=1)

bg_1 = train_features.filter(regex='bg-1')
train_features['bg_one'] = train_features[bg_1.columns].mean(axis=1)

bg_2 = train_features.filter(regex='bg-2')
train_features['bg_two'] = train_features[bg_2.columns].mean(axis=1)

bg_3 = train_features.filter(regex='bg-3')
train_features['bg_three'] = train_features[bg_3.columns].mean(axis=1)

bg_4 = train_features.filter(regex='bg-4')
train_features['bg_four'] = train_features[bg_4.columns].mean(axis=1)

bg_5 = train_features.filter(regex='bg-5')
train_features['bg_five'] = train_features[bg_5.columns].mean(axis=1)

In [144]:
train_features.drop(train_features.filter(regex='bg-').columns, axis=1, inplace=True)

In [149]:
bg_agg_df = train_features.filter(regex='bg')
display(bg_agg_df)

,bg_zero,bg_one,bg_two,bg_three,bg_four,bg_five
0,16.891667,17.958333,14.700000,11.683333,8.450000,9.450000
1,16.266667,17.916667,15.583333,12.658333,8.825000,9.216667
2,15.466667,17.708333,16.600000,13.375000,9.583333,8.783333
3,14.700000,17.375000,17.516667,13.991667,10.575000,8.441667
4,14.158333,16.891667,17.958333,14.700000,11.683333,8.400000
...,...,...,...,...,...,...
177019,8.191667,7.000000,7.725000,8.233333,8.150000,9.716667
177020,8.525000,6.875000,7.683333,8.283333,8.025000,9.750000
177021,8.875000,6.766667,7.666667,8.300000,7.925000,9.741667
177022,9.216667,6.700000,7.650000,8.300000,7.841667,9.716667


In [182]:
insulin_cols = train_features.filter(regex='insulin').columns
train_features[insulin_cols] = train_features[insulin_cols].fillna(0)

In [183]:
insulin_0 = train_features.filter(regex='insulin-0')
train_features['insulin_zero'] = train_features[insulin_0.columns].mean(axis=1)

insulin_1 = train_features.filter(regex='insulin-1')
train_features['insulin_one'] = train_features[insulin_1.columns].mean(axis=1)

insulin_2 = train_features.filter(regex='insulin-2')
train_features['insulin_two'] = train_features[insulin_2.columns].mean(axis=1)

insulin_3 = train_features.filter(regex='insulin-3')
train_features['insulin_three'] = train_features[insulin_3.columns].mean(axis=1)

insulin_4 = train_features.filter(regex='insulin-4')
train_features['insulin_four'] = train_features[insulin_4.columns].mean(axis=1)

insulin_5 = train_features.filter(regex='insulin-5')
train_features['insulin_five'] = train_features[insulin_5.columns].mean(axis=1)


In [186]:
ins_agg_df = train_features.filter(regex='insulin_')
display(ins_agg_df)

,insulin_zero,insulin_one,insulin_two,insulin_three,insulin_four,insulin_five
0,0.055533,0.058300,0.016633,0.008300,0.008300,0.008300
1,0.051383,0.058300,0.029133,0.008300,0.008300,0.008300
2,0.047233,0.058300,0.041633,0.008300,0.008300,0.008300
3,0.043083,0.058300,0.054133,0.008300,0.008300,0.008300
4,0.041700,0.055533,0.058300,0.016633,0.008300,0.008300
...,...,...,...,...,...,...
177019,0.300833,0.037500,1.120833,0.524583,0.039583,0.294583
177020,0.389167,0.033333,1.125000,0.514167,0.045833,0.275833
177021,0.415000,0.047917,1.131250,0.475833,0.084167,0.271667
177022,0.396250,0.064583,1.137500,0.455000,0.105000,0.265417


### Prep and build model

In [190]:
features = bg_agg_df.columns.append(ins_agg_df.columns)
print(features)
X_train = train_features[features]
# y_train set above

Index(['bg_zero', 'bg_one', 'bg_two', 'bg_three', 'bg_four', 'bg_five',
       'insulin_zero', 'insulin_one', 'insulin_two', 'insulin_three',
       'insulin_four', 'insulin_five'],
      dtype='object')


In [191]:
lin_reg = linear_model.LinearRegression()
model = lin_reg.fit(X_train, y_train)

### process test df

In [168]:
bg_cols_test = test.filter(regex='bg').columns
test[bg_cols_test] = test[bg_cols_test].ffill(axis=1).bfill(axis=1)

In [173]:
bg_0 = test.filter(regex='bg-0')
test['bg_zero'] = test[bg_0.columns].mean(axis=1)

bg_1 = test.filter(regex='bg-1')
test['bg_one'] = test[bg_1.columns].mean(axis=1)

bg_2 = test.filter(regex='bg-2')
test['bg_two'] = test[bg_2.columns].mean(axis=1)

bg_3 = test.filter(regex='bg-3')
test['bg_three'] = test[bg_3.columns].mean(axis=1)

bg_4 = test.filter(regex='bg-4')
test['bg_four'] = test[bg_4.columns].mean(axis=1)

bg_5 = test.filter(regex='bg-5')
test['bg_five'] = test[bg_5.columns].mean(axis=1)

In [195]:
insulin_cols_test = test.filter(regex='insulin').columns
test[insulin_cols_test] = test[insulin_cols_test].fillna(0)

In [192]:
insulin_0 = test.filter(regex='insulin-0')
test['insulin_zero'] = test[insulin_0.columns].mean(axis=1)

insulin_1 = test.filter(regex='insulin-1')
test['insulin_one'] = test[insulin_1.columns].mean(axis=1)

insulin_2 = test.filter(regex='insulin-2')
test['insulin_two'] = test[insulin_2.columns].mean(axis=1)

insulin_3 = test.filter(regex='insulin-3')
test['insulin_three'] = test[insulin_3.columns].mean(axis=1)

insulin_4 = test.filter(regex='insulin-4')
test['insulin_four'] = test[insulin_4.columns].mean(axis=1)

insulin_5 = test.filter(regex='insulin-5')
test['insulin_five'] = test[insulin_5.columns].mean(axis=1)

In [196]:
ins_agg_df_test = test.filter(regex='insulin')
display(ins_agg_df_test)

,insulin-5:55,insulin-5:50,insulin-5:45,insulin-5:40,insulin-5:35,insulin-5:30,insulin-5:25,insulin-5:20,insulin-5:15,insulin-5:10,...,insulin-0:15,insulin-0:10,insulin-0:05,insulin-0:00,insulin_zero,insulin_one,insulin_two,insulin_three,insulin_four,insulin_five
0,0.0083,0.0083,0.0083,0.0083,0.0083,0.0083,0.0083,0.0083,0.0083,0.0083,...,0.0417,0.0417,0.0417,0.0417,0.045850,0.227850,0.045800,0.008300,0.183300,0.008300
1,0.0583,0.0583,0.0583,0.0583,0.0583,0.0583,0.0583,0.0417,0.0417,0.0417,...,0.0583,0.0583,0.0583,0.0583,0.308300,0.042500,0.369783,0.048617,0.041700,0.051383
2,0.0583,0.0583,0.0583,0.0583,0.0583,0.0583,0.0583,0.0583,0.0583,0.0583,...,0.0500,0.0500,0.0500,0.0500,0.050000,0.052075,0.058300,0.558300,0.058300,0.058300
3,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0167,0.0167,0.0167,0.0167,...,0.0583,0.0583,0.0583,0.0583,0.033300,0.008300,0.008300,0.008300,0.012500,0.025000
4,4.2333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.0333,0.9083,0.0083,...,0.0000,0.0000,0.0000,0.0000,0.006325,0.015267,0.008300,0.008300,0.058300,0.449967
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3639,0.0771,0.0771,0.0780,0.0833,0.0808,0.0776,0.0792,0.0783,0.0771,0.0771,...,0.0619,0.0562,0.0649,0.0726,0.070058,0.105592,0.051650,0.048733,0.067225,0.078650
3640,0.0202,0.0640,0.0983,0.1117,0.0997,0.0792,0.0338,0.0119,0.0146,0.0458,...,0.0646,0.0646,0.0646,0.0646,0.659858,0.087358,0.045392,0.054142,0.113250,0.066242
3641,0.1855,0.2422,1.9852,0.3492,0.0833,0.0833,0.0833,0.4933,0.0833,0.0833,...,0.0771,0.0771,0.0392,0.0262,0.069700,0.213875,0.179342,0.094358,0.211983,0.319875
3642,0.1144,0.1371,0.1005,0.0771,0.0771,0.0596,0.0277,0.0057,0.0000,0.0195,...,0.0780,0.0857,0.0814,0.0750,0.067375,0.053892,0.070542,0.057108,0.079567,0.062917


In [197]:
ins_agg_df_test.isna().sum()

insulin-5:55     0
insulin-5:50     0
insulin-5:45     0
insulin-5:40     0
insulin-5:35     0
                ..
insulin_one      0
insulin_two      0
insulin_three    0
insulin_four     0
insulin_five     0
Length: 78, dtype: int64

### Make predictions

In [198]:
X_test = test[features]

In [199]:
preds = model.predict(X_test)

In [200]:
test[target] = preds
test.to_csv('submission.csv', columns=['id']+target, index=False)